In [1]:
import os
import pandas as pd
import re

In [2]:
os.chdir("/fs/scratch/PAS0439/Ming/virome_ecology_core_prkaryotes/results/04_prophage_rumen_mags/")

In [3]:
with open("prophage_amr_contigs_final.txt", 'r') as file:
    amr = file.read().split("\n")

In [4]:
amr_out = pd.read_csv("amr/prophage_rumen_mags_category2_card.bln", sep = '\t', names = 'qseqid sseqid pident length qlen slen mismatch gapopen qstart qend sstart send evalue bitscore'.split(' '))

In [5]:
def seperate_id(x):
    out = re.search(r'(.*)_([0-9])+', x)
    contig = out.group(1)
    gene_id = out.group(2)
    return contig, gene_id


In [6]:
for index, row in amr_out.iterrows():
    amr_out.loc[index, ["contig", "gene_id"]] = seperate_id(row["qseqid"])

# Due to multiple hits (similar amr), only choose amr with the highest sequence identity as the true hit

In [7]:
amr_out.loc[amr_out.query('contig in @amr').groupby(['contig',  'gene_id']).pident.idxmax()]

,qseqid,sseqid,pident,length,qlen,slen,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore,contig,gene_id
2624,Bacteria_Denitrobacterium_detoxificans_DSM_218...,gb|CAA10975.1|ARO:3000194|tet(W),96.870,639,640,639,20,0,1,639,1,639,0.000000e+00,1275.0,Bacteria_Denitrobacterium_detoxificans_DSM_218...,1
1322,MAG-4657__scaffold_NODE_6_length_190522_cov_17...,gb|ABA71726.1|ARO:3004253|vanU_in_vanG_cl,53.968,63,68,75,29,0,1,63,1,63,2.410000e-23,81.3,MAG-4657__scaffold_NODE_6_length_190522_cov_17...,0
1317,MAG-4657__scaffold_NODE_6_length_190522_cov_17...,gb|AAY32951.1|ARO:3002837|lnuC,91.463,164,165,164,14,0,1,164,1,164,4.190000e-113,315.0,MAG-4657__scaffold_NODE_6_length_190522_cov_17...,9


In [8]:
genomic_context = pd.read_csv("prophage_amr_for_manual_curation.tsv", sep = "\t")

In [9]:
pd.set_option('display.max_rows', None)

In [10]:
genomic_context.query('fasta in @amr').to_csv('amr_genomic_context_manual_check.tsv', sep = "\t", index = None)